In [150]:
import chess
import chess.svg
from tqdm import tqdm

In [131]:
piece_values = {
    chess.PAWN: 100,
    chess.ROOK: 500,
    chess.KNIGHT: 320,
    chess.BISHOP: 330,
    chess.QUEEN: 900,
    chess.KING: 20000
}
DEPTH_LIMIT = 3

In [51]:
#this functions displays the chess board. May add functions for options
def display_board(board):
    chess.svg.board(board)

In [134]:
def apply_move(board,move_str):
    """
    Applies a move in UCI/long algebraic notation (e.g. 'e2e4', 'e7e8q') 
    to the global chess.Board() instance.
    move_str: str, move in long algebraic/UCI notation
    Returns the updated board.
    """
    move = chess.Move.from_uci(move_str)  # parse the move
    if move in board.legal_moves:
        board.push(move)  # apply it
    else:
        raise ValueError(f"Illegal move: {move_str}")
    return board

In [135]:
chess_board = chess.Board()
NAME = "MUST WIN"
AUTHOR = "IAN WONG"

In [136]:
def handleInput(command):
    parsedCommand = command.split()
    n = len(parsedCommand)
    i = 0
    if command == "quit": return -1
    elif command == "isready": return 0
    elif command == "uci": return 1
    elif command == "go": return 3
    elif parsedCommand[i] == "position":
        print("position")
        i+=1
        if parsedCommand[i] == "fen":
            fen = " ".join(parsedCommand[i+1:i+7])
            chess_board = chess.Board(fen)
            i += 7
        elif parsedCommand[i] == "startpos":
            chess_board = chess.Board()
            i += 1
        if parsedCommand[i] == "moves":
            i+=1
            while i < n:
                chess_board = apply_move(chess_board,parsedCommand[i])
                i+=1
                print(chess_board)
        return 2

In [151]:
def minimax_root(board:chess.Board) -> chess.Move: 
    moves = list(board.legal_moves)
    bestMove, evaluation = moves[0], float("inf")
    for move in tqdm(moves):
        board.push(move)
        score = minimax(board, 0)
        if score < evaluation:
            evaluation = score
            bestMove = move
        board.pop()
    return bestMove
    
 
#uses the minimax algorithm at depth 5 to determine the best move on the chess board
#used the point system example on the blog: https://healeycodes.com/building-my-own-chess-engine
def minimax(board:chess.Board, depth = 0) -> float:
    moves = list(board.legal_moves)
    if depth == DEPTH_LIMIT or board.is_game_over(): 
        return calculate_score(board)
    #pick best score for the opponent
    if depth % 2:
        value = float("-inf")
        for move in moves:
            board.push(move)
            temp = minimax(board,depth+1)
            value = max(value,temp)
            board.pop()
        return value
    else:
        value = float("inf")
        for move in moves:
            board.push(move)
            temp = minimax(board,depth+1)
            value = min(value,temp)
            board.pop()
        return value
    
def determineBestMove(board, depth = 0):
    pass

In [138]:
def handleOutput(command):
    if not command: print("readyok")
    elif command == 1:
        if NAME: print("id name " + NAME)
        if AUTHOR: print("id author " + AUTHOR)
        #add functionality for options that can be changed
        print("uciok")
    elif command == 3:
        move = determineBestMove(chess_board)
        print("bestmove " + move)
        apply_move(chess_board, move)
    
        

In [139]:
def calculate_score(board:chess):
    black_score,white_score = 0,0
    for square in chess.SQUARES:
        at_square = board.piece_at(square)
        if not at_square: continue
        if at_square.color == chess.WHITE:
            white_score += piece_values[at_square.piece_type]
        else:
            black_score += piece_values[at_square.piece_type]
    if board.turn == chess.WHITE: return white_score
    else: return black_score




In [140]:
def main():
    playing = True
    while playing:
        command = input()
        res = handleInput(command)
        handleOutput(res)
        display_board(chess_board)
        if res == -1:
            playing = False


In [143]:
main()

 quit


In [ ]:
board = chess.Board("r1bqk2r/ppp1npbp/1n1pp1p1/8/2PPPP2/2NB1N2/PP4PP/R1BQ1RK1 b kq - 5 8")
while not board.is_checkmate():
    bestMove = minimax_root(board)
    print(bestMove,evaluation)
    board.push(bestMove)
    chess.svg.board(board)
    print(board)


100%|████████████████████████████████████████████████████████████████████████████████| 32/32 [01:05<00:00,  2.04s/it]


g7e5 23900
r . b q k . . r
p p p . n p . p
. n . p p . p .
. . . . b . . .
. . P P P P . .
. . N B . N . .
P P . . . . P P
R . B Q . R K .


100%|████████████████████████████████████████████████████████████████████████████████| 39/39 [00:57<00:00,  1.48s/it]


d1a4 23900
r . b q k . . r
p p p . n p . p
. n . p p . p .
. . . . b . . .
Q . P P P P . .
. . N B . N . .
P P . . . . P P
R . B . . R K .


100%|██████████████████████████████████████████████████████████████████████████████████| 7/7 [00:16<00:00,  2.31s/it]


...

100%|████████████████████████████████████████████████████████████████████████████████| 45/45 [01:41<00:00,  2.26s/it]


f7h8 23900
r . . . k . r N
p b p . n . . p
. . . p p . p .
. q . n . . . .
Q . P P P P . .
. . N B . . . .
P P . . . . P P
R . B . . R K .


100%|████████████████████████████████████████████████████████████████████████████████| 33/33 [01:34<00:00,  2.87s/it]


g8h8 23900
r . . . k . . r
p b p . n . . p
. . . p p . p .
. q . n . . . .
Q . P P P P . .
. . N B . . . .
P P . . . . P P
R . B . . R K .


 23%|██████████████████▋                                                              | 9/39 [00:21<01:17,  2.59s/it]